## Pipeline Demo: Daily Stock Market Data Analytics *(Buildout and documentation in progress)*

#### End-to-end pipeline demonstrating the implementation of the following steps:
1. **Collect data from API and store in data lake**: Python, AWS CLI, AWS S3, AWS EC2 *(In progress)*
    * Download list of <a href="https://www.nasdaq.com/screening/companies-by-industry.aspx" target="blank">NASDAQ companies</a>.
    * Filter out companies for which to collect market data.
    * Query Daily Adjusted Stock Market Time Series <a href="https://www.alphavantage.co/documentation" target="blank">web service</a> for all specified companies.<br><br>
2. **Transform data in data lake** Python, PySpark, AWS EMR (Spark) *(Coming soon)*
3. **Import data into analytics columnar database**: AWS Redshift, SQL *(Coming soon)*
4. **Build pipeline orchestration & scheduling engine**: Python, Apache Airflow *(Coming soon)*
5. **Surface data to public using RESTful web API**: Python, Django *(Coming soon)*

### Get configurations.

Create function to get configuration value from configuration JSON file. By default, configuration file is located in the current working directory and named `configuration.json`.

In [1]:
def getConfigurationValue(configurationKey):

    # Import packages and functions.
    import json, os

    # Get current working directory.
    currentWorkingDirectory = os.getcwd()

    # Load configuration file.
    with open(os.path.join(currentWorkingDirectory, 'configuration.json'), 'r') as configurationFile:
        dictConfigurations = json.load(configurationFile)

    return(dictConfigurations[configurationKey])

**Test:** Print sample configuration values.

In [2]:
print('NASDAQCompaniesSourceURL:', getConfigurationValue('NASDAQCompaniesSourceURL'))
print('NASDAQCompaniesDestinationPathPart:', getConfigurationValue('NASDAQCompaniesDestinationPathPart'))

NASDAQCompaniesSourceURL: https://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download
NASDAQCompaniesDestinationPathPart: Data/NASDAQCompanies.csv


### Programmatically download list of <a href="https://www.nasdaq.com/screening/companies-by-industry.aspx" target="blank">NASDAQ companies</a>.

Create function to download companies CSV into Pandas data frame.

In [3]:
def getNASDAQCompanies(downloadFile = False):
    # Import package(s)/function(s).
    import os, urllib
    import pandas as pd
    
    # Declare variables.

    # Get current working directory.
    currentWorkingDirectory = os.getcwd()

    # Get source URL for NASDAQ companies file.
    NASDAQCompaniesSourceURL = getConfigurationValue('NASDAQCompaniesSourceURL')
    
    # Get destination path part for NASDAQ companies file.
    NASDAQCompaniesDestinationPathPart = getConfigurationValue('NASDAQCompaniesDestinationPathPart')
    
    # Get destination path for NASDAQ companies file.
    NASDAQCompaniesDestinationPath = os.path.abspath(os.path.join(currentWorkingDirectory, NASDAQCompaniesDestinationPathPart))

    # Download data file, if so specified.
    if (downloadFile):
        # Remove destination file, it it exists.
        if(os.path.isfile(NASDAQCompaniesDestinationPath)):
            os.remove(NASDAQCompaniesDestinationPath)

        # Download file.
        downloadedFile, downloadedFileHeaders = urllib.request.urlretrieve(NASDAQCompaniesSourceURL,
                                                                           NASDAQCompaniesDestinationPathPart)

    # Create dfCompanies data frame from NASDAQ companies file.
    dfCompanies = pd.read_csv(NASDAQCompaniesDestinationPathPart)
    
    # Drop emppty last column created as a result of trailing comma.
    dfCompanies.drop(dfCompanies.columns[-1], axis=1, inplace=True)

    return(dfCompanies)

**Test:** Download companies CSV into Pandas data frame.

In [4]:
# dfCompanies = getNASDAQCompanies(downloadFile=True)
dfCompanies = getNASDAQCompanies()

print('dfCompanies count:', len(dfCompanies))

dfCompanies.head(10)

dfCompanies count: 3298


,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote
0,PIH,"1347 Property Insurance Holdings, Inc.",7.15,4.279108e+07,n/a,2014,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih
1,PIHPP,"1347 Property Insurance Holdings, Inc.",24.2,0.000000e+00,n/a,n/a,n/a,n/a,https://www.nasdaq.com/symbol/pihpp
2,TURN,180 Degree Capital Corp.,1.9,5.913097e+07,n/a,n/a,Finance,Finance/Investors Services,https://www.nasdaq.com/symbol/turn
3,FLWS,"1-800 FLOWERS.COM, Inc.",12.05,7.776706e+08,n/a,1999,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws
4,FCCY,1st Constitution Bancorp (NJ),20.2,1.638683e+08,n/a,n/a,Finance,Savings Institutions,https://www.nasdaq.com/symbol/fccy
5,SRCE,1st Source Corporation,49.46,1.329959e+09,n/a,n/a,Finance,Major Banks,https://www.nasdaq.com/symbol/srce
6,VNET,"21Vianet Group, Inc.",7.69,4.737421e+08,61604954,2011,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet
7,TWOU,"2U, Inc.",85.23,4.492967e+09,n/a,2014,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/twou
8,JOBS,"51job, Inc.",85.44,3.113106e+09,36436171,2004,Technology,Diversified Commercial Services,https://www.nasdaq.com/symbol/jobs
9,CAFD,8point3 Energy Partners LP,11.72,9.269192e+08,n/a,2015,Public Utilities,Electric Utilities: Central,https://www.nasdaq.com/symbol/cafd


Create function to filter companies by Symbol or Name.

In [5]:
def getCompaniesByKeyword(dfCompanies, companyKeyword):
    companyKeyword = companyKeyword.lower()
    # Filter dfCompanies by Symbol or Name based on search argument.
    # Use vectorized string methods: lower() and contains()
    dfCompanies = dfCompanies[dfCompanies.Symbol.str.lower().str.contains(companyKeyword)
                              | dfCompanies.Name.str.lower().str.contains(companyKeyword)]
    
    return(dfCompanies)

**Test:** Filter companies by Symbol or Name.

In [10]:
# Filter by 'technologies'.
dfCompaniesFiltered = getCompaniesByKeyword(dfCompanies, 'technologies')

print('dfCompaniesFiltered count:', len(dfCompaniesFiltered))

dfCompaniesFiltered.head(10)

dfCompaniesFiltered count: 67


,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote
50,AEY,"ADDvantage Technologies Group, Inc.",1.34,1.370283e+07,n/a,n/a,Consumer Services,Office Equipment/Supplies/Services,https://www.nasdaq.com/symbol/aey
51,IOTS,Adesto Technologies Corporation,7.25,1.551049e+08,n/a,2015,Technology,Semiconductors,https://www.nasdaq.com/symbol/iots
62,ADVM,"Adverum Biotechnologies, Inc.",6.4,3.979090e+08,n/a,2014,Health Care,Biotechnology: Biological Products (No Diagnos...,https://www.nasdaq.com/symbol/advm
89,AKAM,"Akamai Technologies, Inc.",70.19,1.193452e+10,n/a,1999,Miscellaneous,Business Services,https://www.nasdaq.com/symbol/akam
95,AKTS,"Akoustis Technologies, Inc.",6.64,1.482094e+08,n/a,n/a,Public Utilities,Telecommunications Equipment,https://www.nasdaq.com/symbol/akts
115,AMOT,"Allied Motion Technologies, Inc.",39.08,3.684065e+08,n/a,n/a,Capital Goods,Electrical Products,https://www.nasdaq.com/symbol/amot
153,AETI,"American Electric Technologies, Inc.",1.0213,8.854314e+06,n/a,n/a,Energy,Industrial Machinery/Components,https://www.nasdaq.com/symbol/aeti
200,AGTC,Applied Genetic Technologies Corporation,4.2,7.604058e+07,n/a,2014,Health Care,Biotechnology: Biological Products (No Diagnos...,https://www.nasdaq.com/symbol/agtc
209,AQB,"AquaBounty Technologies, Inc.",3.1,3.905551e+07,n/a,n/a,n/a,n/a,https://www.nasdaq.com/symbol/aqb
306,ACLS,"Axcelis Technologies, Inc.",25.8,8.289235e+08,n/a,2000,Technology,Industrial Machinery/Components,https://www.nasdaq.com/symbol/acls


### Query Alpha Advantage's Daily Adjusted Stock Market Time Series <a href="https://www.alphavantage.co/documentation/#dailyadj" target="blank">web service</a> for all specified companies.

Create function to query web service to load daily adjusted time series for specified stock symbol into Pandas data frame. Optionally, save service response as JSON file.

In [7]:
def getAlphaAdvantageDailyAdjustedStockMarketTimeSeries(ticker, downloadFile = False):
    # Import package(s)/function(s).
    import os, json, requests
    import pandas as pd
    
    # Declare variables.

    # Get current working directory.
    currentWorkingDirectory = os.getcwd()

    # Get API Key.
    APIKey = getConfigurationValue('AlphaVantageAPIKey')
    
    # Get service request URL template.
    serviceRequestURLTemplate = getConfigurationValue('AlphaVantageTimeSeriesDailyAdjustedServiceRequestURLTemplate')
    
    # Get service request URL.
    serviceRequestURL = serviceRequestURLTemplate.replace('$symbol$', ticker).replace('$apiKey$', APIKey)
    
    # Get destination path part.
    destinationPathPart = getConfigurationValue('AlphaAdvantageTimeSeriesDailyAdjustedDestinationPathPart')

    # Get destination path.
    destinationPath = os.path.abspath(os.path.join(currentWorkingDirectory, destinationPathPart, ticker + '.json'))

    # Issue request to service. Get response.
    response = requests.get(serviceRequestURL)
    
    # Convert response to JSON format.
    JSONResponse = response.json()

    # Get time series dictionary from JSON.
    dictResponse = JSONResponse['Time Series (Daily)']
    
    # Specify column order to use for data frame.
    columns = ['open', 'high', 'low', 'close', 'adjustedClose', 'volume', 'dividend', 'split']
    
    # Construct data frame from dictionary.
    # Rename columns.
    # Use specified column order.
    dfMarketData = (pd.DataFrame
                    .from_dict(dictResponse, orient='index')
                    .rename(columns={'1. open':'open',
                                     '2. high':'high',
                                     '3. low':'low',
                                     '4. close':'close',
                                     '5. adjusted close':'adjustedClose',
                                     '6. volume':'volume',
                                     '7. dividend amount':'dividend',
                                     '8. split coefficient':'split'})
                   [columns])
    
    # Download data file, if so specified.
    if (downloadFile):
        # Remove destination file, it it exists.
        if(os.path.isfile(destinationPath)):
            os.remove(destinationPath)
            
        with open(destinationPath, 'w') as fileDestinationPath:
            json.dump(JSONResponse, fileDestinationPath, ensure_ascii=False, indent=0, sort_keys=True)

    # Return Pandas data frame.
    return(dfMarketData)

**Test:** Query web service to get daily adjusted time series for `Amazon`.

In [8]:
#getAlphaAdvantageDailyAdjustedStockMarketTimeSeries('AMZN', downloadFile=True)
getAlphaAdvantageDailyAdjustedStockMarketTimeSeries('AMZN').head(10)

,open,high,low,close,adjustedClose,volume,dividend,split
2000-01-03,81.5000,89.5600,80.0000,89.3800,89.3800,16117600,0.0000,1.0000
2000-01-04,85.3800,91.5000,81.7500,81.9400,81.9400,17487400,0.0000,1.0000
2000-01-05,70.7500,75.1300,69.6300,71.7500,71.7500,38457400,0.0000,1.0000
2000-01-06,71.3100,72.6900,64.0000,65.5600,65.5600,18752000,0.0000,1.0000
2000-01-07,67.0000,70.5000,66.1900,69.5600,69.5600,10505400,0.0000,1.0000
2000-01-10,72.5600,72.6300,65.5600,69.1900,69.1900,14757900,0.0000,1.0000
2000-01-11,66.8800,70.0000,65.0000,66.7500,66.7500,10532700,0.0000,1.0000
2000-01-12,67.8800,68.0000,63.0000,63.5600,63.5600,10804500,0.0000,1.0000
2000-01-13,64.9400,67.1900,63.1300,65.9400,65.9400,10448100,0.0000,1.0000
2000-01-14,66.7500,68.5000,64.0000,64.2500,64.2500,6853600,0.0000,1.0000


**Test:** Query web service to get daily adjusted time series for `Google`.

In [9]:
#getAlphaAdvantageDailyAdjustedStockMarketTimeSeries('GOOG', downloadFile=True)
getAlphaAdvantageDailyAdjustedStockMarketTimeSeries('GOOG').head(10)

,open,high,low,close,adjustedClose,volume,dividend,split
2014-03-27,568.0000,568.0000,552.9200,558.4600,558.4600,13100,0.0000,1.0000
2014-03-28,561.2000,566.4300,558.6700,559.9900,559.9900,41100,0.0000,1.0000
2014-03-31,566.8900,567.0000,556.9300,556.9700,556.9700,10800,0.0000,1.0000
2014-04-01,558.7100,568.4500,558.7100,567.1600,567.1600,7900,0.0000,1.0000
2014-04-02,565.1060,604.8300,562.1900,567.0000,567.0000,146700,0.0000,1.0000
2014-04-03,569.8500,587.2800,564.1300,569.7400,569.7400,5085200,0.0000,1.0000
2014-04-04,574.6500,577.7700,543.0000,543.1400,543.1400,6351900,0.0000,1.0000
2014-04-07,540.7400,548.4800,527.1500,538.1500,538.1500,4389600,0.0000,1.0000
2014-04-08,542.6000,555.0000,541.6100,554.9000,554.9000,3142600,0.0000,1.0000
2014-04-09,559.6200,565.3700,552.9500,564.1400,564.1400,3321700,0.0000,1.0000
